In [28]:
# import dependencies
#import os
import spotipy
import spotipy.util as util
import pandas as pd
import pprint
##import csv


#Client_ID = "7655188a18144343b07e44965cc38310"
#Client_Secret = "2bec9b47853e4ea3a419418a211ec45d"

## Grab csv files dataframe

In [145]:
top_2017 = pd.read_csv("assets/data/top2017.csv", encoding='latin-1')
top_2018 = pd.read_csv("assets/data/top2018.csv", encoding='unicode_escape')
top_2019 = pd.read_csv("assets/data/top2019.csv", encoding='unicode_escape')

### create search list of each year

In [121]:
# initialize list to scrape
search_data = []

# pull tracks and artists
tracks = list(top_2019['Track.Name'])
artists = list(top_2019['Artist.Name'])
# add each list to search_data
for i in range(len(tracks)):
    search_data.append([tracks[i], artists[i]])

search_data

[['Se\x81EEorita', 'Shawn Mendes'],
 ['China', 'Anuel AA'],
 ['boyfriend (with Social House)', 'Ariana Grande'],
 ['Beautiful People (feat. Khalid)', 'Ed Sheeran'],
 ['Goodbyes (Feat. Young Thug)', 'Post Malone'],
 ["I Don't Care (with Justin Bieber)", 'Ed Sheeran'],
 ['Ransom', 'Lil Tecca'],
 ['How Do You Sleep?', 'Sam Smith'],
 ['Old Town Road - Remix', 'Lil Nas X'],
 ['bad guy', 'Billie Eilish'],
 ['Callaita', 'Bad Bunny'],
 ['Loco Contigo (feat. J. Balvin & Tyga)', 'DJ Snake'],
 ['Someone You Loved', 'Lewis Capaldi'],
 ['Otro Trago - Remix', 'Sech'],
 ['Money In The Grave (Drake ft. Rick Ross)', 'Drake'],
 ['No Guidance (feat. Drake)', 'Chris Brown'],
 ['LA CANCIÓN', 'J Balvin'],
 ['Sunflower - Spider-Man: Into the Spider-Verse', 'Post Malone'],
 ['Lalala', 'Y2K'],
 ['Truth Hurts', 'Lizzo'],
 ['Piece Of Your Heart', 'MEDUZA'],
 ['Panini', 'Lil Nas X'],
 ['No Me Conoce - Remix', 'Jhay Cortez'],
 ['Soltera - Remix', 'Lunay'],
 ['bad guy (with Justin Bieber)', 'Billie Eilish'],
 ["If 

In [127]:
def search_by_track(track, artist):

    # create spotipy object
    sp = spotipy.Spotify(auth=token)
    # search by track name 
    result = sp.search(q=track, type='track')
    # save first 10 results
    first_ten_albums = result['tracks']['items']
    # loop through each album 
    for album in first_ten_albums:
        # extract artist of each album
        possible_artist = album['album']['artists'][0]['name']
        # see if album artist matches query artist
        if possible_artist == artist:
            # match found, add to song-ids
            value = [artist, track, result['tracks']['items'][0]['id']]
            return value
    return None

def search_by_artist(artist, track):
    ''' returns correct track by artist to search'''
    # create spotipy object
    sp = spotipy.Spotify(auth=token)
  
    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    top_tracks = sp.artist_top_tracks(artist_id)
    top_10 = top_tracks['tracks']
    # loop through top tracks, check to see if any match track query
    for song in top_10:
        possible_song = song['name']
        # matches enough, match found
        if track[:2] == possible_song[:2]:
            track = possible_song
            return track
    return None

def grab_genre(artist):
    ''' returns a genre of artist'''
    # create spotipy object
    sp = spotipy.Spotify(auth=token)

    # search by artist name
    result = sp.search(q=artist, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    genre = sp.artist(artist_id)['genres']
    return genre
 

In [134]:
#https://open.spotify.com/user/1298997744?si=ijwp9NZOQoyjO9qOZliclw
SPOTIPY_CLIENT_ID='7655188a18144343b07e44965cc38310'
SPOTIPY_CLIENT_SECRET='2bec9b47853e4ea3a419418a211ec45d'
SPOTIPY_REDIRECT_URI='https://google.com/'
scope = 'user-library-read'
# Grab username
username = "1298997744"

token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

results = []

# Create our spotify object
for query in search_data:
    if token:
        # create track & artist search variables
        track = query[0]
        artist = query[1]
        genre = grab_genre(artist)

        # run search by track to find results
        value = search_by_track(track, artist)

        # if search_by_track returns none, use search_by_artist.
        # else append to results
        if value == None:
            correct_track = search_by_artist(artist, track)
            # run search_by_track again
            value = search_by_track(correct_track, artist)
            value.append(genre)
            results.append(value)
        else:
            value.append(genre)
            results.append(value)   
    else:
        print("Can't get token for", username)

In [135]:
results

[['Shawn Mendes',
  'Señorita',
  '0TK2YIli7K1leLovkQiNik',
  ['canadian pop', 'pop', 'post-teen pop', 'viral pop']],
 ['Anuel AA',
  'China',
  '2ksOAxtIxY8yElEWw8RhgK',
  ['latin', 'reggaeton flow', 'trap latino']],
 ['Ariana Grande',
  'boyfriend (with Social House)',
  '0Ryd8975WihbObpp5cPW1t',
  ['dance pop', 'pop', 'post-teen pop']],
 ['Ed Sheeran',
  'Beautiful People (feat. Khalid)',
  '4evmHXcjt3bTUHD1cvny97',
  ['pop', 'uk pop']],
 ['Post Malone',
  'Goodbyes (Feat. Young Thug)',
  '0t3ZvGKlmYmVsDzBJAXK8C',
  ['dfw rap', 'melodic rap', 'rap']],
 ['Ed Sheeran',
  "I Don't Care (with Justin Bieber)",
  '3HVWdVOQ0ZA45FuZGSfvns',
  ['pop', 'uk pop']],
 ['Lil Tecca', 'Ransom', '1lOe9qE0vR9zwWQAOk6CoO', ['melodic rap']],
 ['Sam Smith',
  'How Do You Sleep?',
  '6b2RcmUt1g9N9mQ3CbjX2Y',
  ['pop', 'uk pop']],
 ['Lil Nas X',
  'Old Town Road - Remix',
  '2YpeDb67231RjR0MgVLzsG',
  ['country rap', 'lgbtq+ hip hop', 'pop rap']],
 ['Billie Eilish', 'bad guy', '2Fxmhks0bxGSBdJ92vM42m', ['

In [136]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

# create spotipy object
sp = spotipy.Spotify(auth=token)

top_2019_data = []
for each in results:
    features = sp.audio_features(each[2])
    data = features[0]
    data['name'] = each[1]
    data['artists'] = each[0]
    data['year'] = '2019'
    data['genre'] = each[3]
    top_2019_data.append(data)


top_2019_data

[{'danceability': 0.759,
  'energy': 0.54,
  'key': 9,
  'loudness': -6.039,
  'mode': 0,
  'speechiness': 0.0287,
  'acousticness': 0.037,
  'instrumentalness': 0,
  'liveness': 0.0945,
  'valence': 0.75,
  'tempo': 116.947,
  'type': 'audio_features',
  'id': '0TK2YIli7K1leLovkQiNik',
  'uri': 'spotify:track:0TK2YIli7K1leLovkQiNik',
  'track_href': 'https://api.spotify.com/v1/tracks/0TK2YIli7K1leLovkQiNik',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0TK2YIli7K1leLovkQiNik',
  'duration_ms': 190960,
  'time_signature': 4,
  'name': 'Señorita',
  'artists': 'Shawn Mendes',
  'year': '2019',
  'genre': ['canadian pop', 'pop', 'post-teen pop', 'viral pop']},
 {'danceability': 0.786,
  'energy': 0.808,
  'key': 7,
  'loudness': -3.702,
  'mode': 1,
  'speechiness': 0.0882,
  'acousticness': 0.0846,
  'instrumentalness': 0.000289,
  'liveness': 0.0822,
  'valence': 0.609,
  'tempo': 105.027,
  'type': 'audio_features',
  'id': '2ksOAxtIxY8yElEWw8RhgK',
  'uri': 'spotify:t

In [137]:
top2019_df = pd.DataFrame.from_dict(top_2019_data)
top_2019_df = top2019_df.drop(columns=['type', 'uri','track_href','analysis_url'])
##df = df[['mean', '0', '1', '2', '3']]
top_2019_df = top_2019_df[['id','name','artists','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature','year','genre']]
top_2019_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
0,0TK2YIli7K1leLovkQiNik,Señorita,Shawn Mendes,0.759,0.540,9,-6.039,0,0.0287,0.03700,0.000000,0.0945,0.750,116.947,190960,4,2019,"[canadian pop, pop, post-teen pop, viral pop]"
1,2ksOAxtIxY8yElEWw8RhgK,China,Anuel AA,0.786,0.808,7,-3.702,1,0.0882,0.08460,0.000289,0.0822,0.609,105.027,301714,4,2019,"[latin, reggaeton flow, trap latino]"
2,0Ryd8975WihbObpp5cPW1t,boyfriend (with Social House),Ariana Grande,0.400,0.795,10,-3.731,0,0.4610,0.11900,0.000000,0.1590,0.702,190.097,186107,4,2019,"[dance pop, pop, post-teen pop]"
3,4evmHXcjt3bTUHD1cvny97,Beautiful People (feat. Khalid),Ed Sheeran,0.640,0.648,5,-8.113,0,0.1870,0.12400,0.000000,0.0802,0.548,92.977,197867,4,2019,"[pop, uk pop]"
4,0t3ZvGKlmYmVsDzBJAXK8C,Goodbyes (Feat. Young Thug),Post Malone,0.548,0.653,5,-3.875,1,0.0810,0.45600,0.000000,0.1030,0.163,150.068,174853,4,2019,"[dfw rap, melodic rap, rap]"
5,3HVWdVOQ0ZA45FuZGSfvns,I Don't Care (with Justin Bieber),Ed Sheeran,0.798,0.675,6,-5.041,1,0.0442,0.09120,0.000000,0.0894,0.842,101.956,219947,4,2019,"[pop, uk pop]"
6,1lOe9qE0vR9zwWQAOk6CoO,Ransom,Lil Tecca,0.745,0.642,7,-6.257,0,0.2870,0.02040,0.000000,0.0658,0.226,179.974,131240,4,2019,[melodic rap]
7,6b2RcmUt1g9N9mQ3CbjX2Y,How Do You Sleep?,Sam Smith,0.477,0.682,1,-4.931,0,0.0925,0.15300,0.000000,0.0763,0.345,110.567,202205,5,2019,"[pop, uk pop]"
8,2YpeDb67231RjR0MgVLzsG,Old Town Road - Remix,Lil Nas X,0.878,0.619,6,-5.560,1,0.1020,0.05330,0.000000,0.1130,0.639,136.041,157067,4,2019,"[country rap, lgbtq+ hip hop, pop rap]"
9,2Fxmhks0bxGSBdJ92vM42m,bad guy,Billie Eilish,0.701,0.425,7,-10.965,1,0.3750,0.32800,0.130000,0.1000,0.562,135.128,194088,4,2019,"[electropop, pop]"


In [21]:
#top_2019_df.to_csv('assets/data/top2019fixed.csv', index=False)
top_2019_df.to_excel('assets/data/top2019fixed2.xlsx', index=False)

In [22]:
final_df = pd.read_excel("assets/data/top2019fixed2.xlsx")
final_df

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
0,0TK2YIli7K1leLovkQiNik,Señorita,Shawn Mendes,0.759,0.540,9,-6.039,0,0.0287,0.03700,0.000000,0.0945,0.750,116.947,190960,4,2019
1,2ksOAxtIxY8yElEWw8RhgK,China,Anuel AA,0.786,0.808,7,-3.702,1,0.0882,0.08460,0.000289,0.0822,0.609,105.027,301714,4,2019
2,0Ryd8975WihbObpp5cPW1t,boyfriend (with Social House),Ariana Grande,0.400,0.795,10,-3.731,0,0.4610,0.11900,0.000000,0.1590,0.702,190.097,186107,4,2019
3,4evmHXcjt3bTUHD1cvny97,Beautiful People (feat. Khalid),Ed Sheeran,0.640,0.648,5,-8.113,0,0.1870,0.12400,0.000000,0.0802,0.548,92.977,197867,4,2019
4,0t3ZvGKlmYmVsDzBJAXK8C,Goodbyes (Feat. Young Thug),Post Malone,0.548,0.653,5,-3.875,1,0.0810,0.45600,0.000000,0.1030,0.163,150.068,174853,4,2019
5,3HVWdVOQ0ZA45FuZGSfvns,I Don't Care (with Justin Bieber),Ed Sheeran,0.798,0.675,6,-5.041,1,0.0442,0.09120,0.000000,0.0894,0.842,101.956,219947,4,2019
6,1lOe9qE0vR9zwWQAOk6CoO,Ransom,Lil Tecca,0.745,0.642,7,-6.257,0,0.2870,0.02040,0.000000,0.0658,0.226,179.974,131240,4,2019
7,6b2RcmUt1g9N9mQ3CbjX2Y,How Do You Sleep?,Sam Smith,0.477,0.682,1,-4.931,0,0.0925,0.15300,0.000000,0.0763,0.345,110.567,202205,5,2019
8,2YpeDb67231RjR0MgVLzsG,Old Town Road - Remix,Lil Nas X,0.878,0.619,6,-5.560,1,0.1020,0.05330,0.000000,0.1130,0.639,136.041,157067,4,2019
9,2Fxmhks0bxGSBdJ92vM42m,bad guy,Billie Eilish,0.701,0.425,7,-10.965,1,0.3750,0.32800,0.130000,0.1000,0.562,135.128,194088,4,2019


### finds genre of artist

In [46]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)

artist_genres = {}

for person in artists:
    # create spotipy object
    sp = spotipy.Spotify(auth=token)

    # search by artist name
    result = sp.search(q=person, type='artist')
    # grab artist id
    artist_id = result['artists']['items'][0]['id']
    # search artist top tracks
    genre = sp.artist(artist_id)['genres']
    if artist_genres.get(person) == None:
        artist_genres[person] = genre
 


In [47]:
artist_genres

{'Shawn Mendes': ['canadian pop', 'pop', 'post-teen pop', 'viral pop'],
 'Anuel AA': ['latin', 'reggaeton flow', 'trap latino'],
 'Ariana Grande': ['dance pop', 'pop', 'post-teen pop'],
 'Ed Sheeran': ['pop', 'uk pop'],
 'Post Malone': ['dfw rap', 'melodic rap', 'rap'],
 'Lil Tecca': ['melodic rap'],
 'Sam Smith': ['pop', 'uk pop'],
 'Lil Nas X': ['country rap', 'lgbtq+ hip hop', 'pop rap'],
 'Billie Eilish': ['electropop', 'pop'],
 'Bad Bunny': ['latin', 'reggaeton', 'trap latino'],
 'DJ Snake': ['electronic trap'],
 'Lewis Capaldi': ['pop', 'uk pop'],
 'Sech': ['panamanian pop', 'reggaeton'],
 'Drake': ['canadian hip hop',
  'canadian pop',
  'hip hop',
  'pop rap',
  'rap',
  'toronto rap'],
 'Chris Brown': ['dance pop', 'pop', 'pop rap', 'r&b', 'rap'],
 'J Balvin': ['latin', 'reggaeton'],
 'Y2K': ['pop rap'],
 'Lizzo': ['escape room', 'minnesota hip hop', 'pop', 'pop rap', 'trap queen'],
 'MEDUZA': ['pop house'],
 'Jhay Cortez': ['latin', 'reggaeton', 'reggaeton flow', 'trap latino

In [264]:
def add_genre(dataframe):
    '''takes in a dataframe, adds a genre column by artist'''
    token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
    
    # grab artist column for Spotipy search
    artists = list(dataframe['artists'])  
   
    # initialize genre column
    genres = []

    for person in artists:
        # create spotipy object
        sp = spotipy.Spotify(auth=token)
        # search by artist name
        result = sp.search(q=person, type='artist')
        try:
            # grab artist id
            artist_id = result['artists']['items'][0]['id']
            # search artist top tracks
            genre = sp.artist(artist_id)['genres']
            genres.append(genre)
        except IndexError:
            print(person)
            i = dataframe.index[dataframe['artists'] == person]
            song_df = dataframe.loc[i, 'name']
            song_list = song_df.values
            song = song_list[0]
            result = sp.search(q=song, type='track')
            artist = result['tracks']['items'][0]['artists'][0]['name']
            result = sp.search(q=artist, type='artist')
            genre = result['artists']['items'][0]['genres']
            genres.append(genre)
                   
    dataframe['genre'] = genres


In [265]:
add_genre(top_2018)

Ti?sto
retrying ...1secs


In [266]:
top_2018[20:70]

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,genre
20,5CLGzJsGqhCEECcpnFQA8,"These Days (feat. Jess Glynne, Macklemore & Da...",Rudimental,0.653,0.809,0,-4.057,1,0.0474,0.19400,0.000000,0.1650,0.5500,92.213,210773,4,2018,"[dance pop, edm, liquid funk, pop, tropical ho..."
21,3V8UKqhEK5zBkBb6d6ub8,Te Bot? - Remix,Nio Garcia,0.903,0.675,11,-3.445,0,0.2140,0.54200,0.000013,0.0595,0.4420,96.507,417920,4,2018,"[latin, latin hip hop, perreo, reggaeton, regg..."
22,7uzmGiiJyRfuViKKK3lVm,Mine,Bazzi,0.710,0.789,4,-3.874,1,0.0722,0.01610,0.000003,0.4510,0.7170,142.929,131064,4,2018,"[pop, post-teen pop]"
23,2iUXsYOEPhVqEBwsqP70r,Youngblood,5 Seconds of Summer,0.596,0.854,7,-5.114,0,0.4630,0.01690,0.000000,0.1240,0.1520,120.274,203418,4,2018,"[boy band, dance pop, pop, post-teen pop]"
24,2xJCMIJfcNYDc5iR0sAm2,New Rules,Dua Lipa,0.762,0.700,9,-6.021,0,0.0694,0.00261,0.000016,0.1530,0.6080,116.073,209320,4,2018,"[dance pop, pop, post-teen pop, uk pop]"
25,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.58100,0.000000,0.0931,0.9310,95.977,233713,4,2018,"[pop, uk pop]"
26,45Egmo7icyopuzJN0oMEd,Love Lies (with Normani),Khalid,0.708,0.648,6,-5.626,1,0.0449,0.09560,0.000000,0.1340,0.3380,143.955,201707,4,2018,"[alternative r&b, pop]"
27,4e4fqjx0Izh4svvTef1z7,Meant to Be (feat. Florida Georgia Line),Bebe Rexha,0.642,0.772,10,-6.610,1,0.0848,0.04760,0.000000,0.0646,0.5890,153.995,164205,4,2018,"[dance pop, pop, post-teen pop]"
28,7m9OqQk4RVRkw9JJdeAw9,Jocelyn Flores,XXXTENTACION,0.872,0.391,0,-9.144,0,0.2420,0.46900,0.000004,0.2970,0.4370,134.021,119133,4,2018,"[emo rap, miami hip hop]"
29,0tgVpDi06FyKpA1z0VMD4,Perfect,Ed Sheeran,0.599,0.448,8,-6.312,1,0.0232,0.16300,0.000000,0.1060,0.1680,95.050,263400,3,2018,"[pop, uk pop]"


In [261]:
token = util.prompt_for_user_token(username, scope, client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri=SPOTIPY_REDIRECT_URI)
song_id = '6DCZcSspjsKoFjzjrWoCd'
track = 'Jackie Chan'
sp = spotipy.Spotify(auth=token)

result = sp.search(q=track, type='track')
artist = result['tracks']['items'][0]['artists'][0]['name']
result = sp.search(q=artist, type='artist')
result['artists']['items'][0]['genres']

['big room',
 'brostep',
 'dance pop',
 'edm',
 'electro house',
 'house',
 'trance',
 'tropical house']

In [212]:
top_2018.loc[45, 'name']

'Jackie Chan'